# LLM agent Hackathon 
### Alireza Ghafarollahi, MIT

## Topic: A pre-programmed multi-agent system with Python

In [14]:
import json
import os
import re
import openai
from openai import OpenAI
import json
import os.path as osp
import subprocess

print(f'openai version: {openai.__version__}')

openai version: 1.76.0


## config.json contains your openai api key

In [19]:
with open('config.json') as f:
    config = json.load(f)

OPENAI_API_KEY = config["api_key"]
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [28]:
from openai import OpenAI

def get_response_from_llm(model: str, messages: list, temperature: float = 0.7, max_completion_tokens: int = 500) -> str:
    """
    Sends a prompt to an OpenAI chat model and returns the generated response content.

    Parameters:
    - model (str): The name of the OpenAI model to use (e.g., "gpt-4", "gpt-4o", "gpt-3.5-turbo").
    - messages (list): A list of message dicts in the format [{"role": "developer"|"user"|"assistant", "content": "text"}].
    - temperature (float): Controls randomness in the output. Higher values (e.g., 1.0) make output more random, lower values (e.g., 0.2) more focused.
    - max_completion_tokens (int): Maximum number of tokens in the response.

    Returns:
    - str: The content of the assistant's reply.
    """
    client = OpenAI()
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_completion_tokens,
    )
    
    content = response.choices[0].message.content
    return content

In [29]:
response = get_response_from_llm(
    model='gpt-4o',
    messages=[{"role": "user", "content": """You are a scientist. Briefly and accurately respond to the following question:
What are the current hot trends in AI?"""}],
    temperature=1,
              )

In [31]:
print(response)

As of 2023, several hot trends in AI include:

1. **Generative AI**: AI models, particularly large language models (LLMs) and generative adversarial networks (GANs), are gaining traction for their ability to create text, images, music, and even video content. OpenAI's GPT series and DALL-E are prominent examples.

2. **AI in Healthcare**: AI is increasingly being used for drug discovery, personalized medicine, diagnostic imaging, and predictive analytics for patient outcomes.

3. **Edge AI and IoT**: Deploying AI at the edge, such as on IoT devices and smartphones, facilitates real-time processing with lower latency and improved privacy.

4. **Explainability and Trustworthy AI**: As AI systems are integrated into critical applications, there is a growing focus on explainable AI (XAI) to ensure transparency and trust.

5. **AI Ethics and Regulation**: There is an increasing emphasis on developing ethical guidelines and regulations to ensure AI is used responsibly and does not cause harm

## Adding msg_history to keep track of previous converstations

In [40]:
def get_response_from_llm(model: str, messages: list, msg_history=None, temperature: float = 0.7, max_completion_tokens: int = 500) -> str:
    """
    Sends a prompt to an OpenAI chat model and returns the generated response content.

    Parameters:
    - model (str): The name of the OpenAI model to use (e.g., "gpt-4", "gpt-4o", "gpt-3.5-turbo").
    - messages (list): A list of message dicts in the format [{"role": "developer"|"user"|"assistant", "content": "text"}].
    - temperature (float): Controls randomness in the output. Higher values (e.g., 1.0) make output more random, lower values (e.g., 0.2) more focused.
    - max_completion_tokens (int): Maximum number of tokens in the response.

    Returns:
    - str: The content of the assistant's reply.
    """
    if msg_history is None:
        msg_history = []

    new_msg_history = msg_history + messages
    
    client = OpenAI()
    response = client.chat.completions.create(
        model=model,
        messages=new_msg_history,
        temperature=temperature,
        max_tokens=max_completion_tokens,
    )
    
    content = response.choices[0].message.content
    new_msg_history = new_msg_history + [{"role": "assistant", "content": content}]
    
    return content, new_msg_history

In [41]:
messages = [{"role": "user", "content": """You are a scientist. Briefly and accurately respond to the following question:
What are the current hot trends in AI?"""}]

response, msg_history = get_response_from_llm(
    model='gpt-4o',
    messages=messages,
    msg_history=None,
    temperature=1,
              )

In [42]:
msg_history

[{'role': 'user',
  'content': 'You are a scientist. Briefly and accurately respond to the following question:\nWhat are the current hot trends in AI?'},
 {'role': 'assistant',
  'content': "As of 2023, several hot trends in AI include:\n\n1. **Generative AI**: Technologies like Generative Adversarial Networks (GANs) and transformers have enabled advancements in creating high-resolution images, music, and text, with applications from art to synthetic data generation.\n\n2. **Large Language Models (LLMs)**: These models, such as GPT and its successors, continue to improve in natural language understanding and generation, with applications in chatbots, content creation, and translation.\n\n3. **AI in Healthcare**: AI is increasingly used for personalized medicine, drug discovery, and diagnostic tools, leveraging vast amounts of medical data to improve patient outcomes.\n\n4. **Ethical and Explainable AI**: There is growing focus on developing AI systems that are transparent, unbiased, an

In [43]:
messages = [{"role": "user", "content": """You are a scientist. Summarize the text that was just created."""}]

response, msg_history = get_response_from_llm(
    model='gpt-4o',
    messages=messages,
    msg_history=msg_history,
    temperature=1,
              )

In [44]:
msg_history

[{'role': 'user',
  'content': 'You are a scientist. Briefly and accurately respond to the following question:\nWhat are the current hot trends in AI?'},
 {'role': 'assistant',
  'content': "As of 2023, several hot trends in AI include:\n\n1. **Generative AI**: Technologies like Generative Adversarial Networks (GANs) and transformers have enabled advancements in creating high-resolution images, music, and text, with applications from art to synthetic data generation.\n\n2. **Large Language Models (LLMs)**: These models, such as GPT and its successors, continue to improve in natural language understanding and generation, with applications in chatbots, content creation, and translation.\n\n3. **AI in Healthcare**: AI is increasingly used for personalized medicine, drug discovery, and diagnostic tools, leveraging vast amounts of medical data to improve patient outcomes.\n\n4. **Ethical and Explainable AI**: There is growing focus on developing AI systems that are transparent, unbiased, an